In [ ]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from datetime import date
import joblib

In [ ]:
step = 5
btc = pd.read_csv('btc-usdt.csv')

btc.drop(['Open Time','Close Time','Ignore','TB Base Volume','TB Quote Volume','Quote Asset Volume'], axis = 1, inplace = True)
train = btc.loc[:39999]
valid = btc.loc[40000:]

scaler = MinMaxScaler(feature_range=(0, 1))  

scaled_data = scaler.fit_transform(train)
x_train, y_train = [], []
for i in range(step, len(train)):
    x_train.append(scaled_data[i - step:i])
    y_train.append(scaled_data[i, 3])

x_train, y_train = np.array(x_train), np.array(y_train)

scaled_data = scaler.fit_transform(valid)
x_valid, y_valid = [], []
for i in range(step, len(valid)):
    x_valid.append(scaled_data[i - step:i])
    y_valid.append(scaled_data[i, 3])

x_valid, y_valid = np.array(x_valid), np.array(y_valid)
x_train.shape

In [ ]:
''' = joblib.load('filename.pkl') '''

In [ ]:
epochs = 30
batch_size = 16
model = Sequential()
model.add(LSTM(units=150, return_sequences=True, input_dim=x_train.shape[-1], input_length=x_train.shape[1]))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train , epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
joblib.dump(model, 'laptop.pkl')

In [ ]:
closing_price = model.predict(x_valid)
scaler.fit_transform(pd.DataFrame(valid['Close'].values))

closing_price = scaler.inverse_transform(closing_price)
real = scaler.inverse_transform([y_valid])
print(real)
# print(closing_price)

print(mean_squared_error(real.reshape(-1,1), closing_price.reshape(-1,1)))
print(closing_price.shape)
print(real.shape)

In [ ]:
plt.figure(figsize=(15, 10))
dict_data = {
    'Predictions': closing_price.reshape(1,-1)[0,:],
    'Close': real[0,:]
}
data_pd = pd.DataFrame(dict_data)

plt.plot(data_pd[['Close', 'Predictions']])
plt.show()

In [ ]:
eth = pd.read_csv('eth-usdt.csv')
eth.drop(['Open Time','Close Time','Ignore','TB Base Volume','TB Quote Volume','Quote Asset Volume'], axis = 1, inplace = True)

newscaler = MinMaxScaler(feature_range=(0, 1))
scaled_eth = newscaler.fit_transform(eth[:])
x_eth, y_eth = [], []
for i in range(step, len(eth)):
    x_eth.append(scaled_eth[i - step:i])
    y_eth.append(scaled_eth[i, 3])

x_eth, y_eth = np.array(x_eth), np.array(y_eth)

pred = model.predict(x_eth)
newscaler.fit_transform(pd.DataFrame(eth['Close'].values))
pred = newscaler.inverse_transform(pred)
re = newscaler.inverse_transform([y_eth])
print(mean_squared_error(re.reshape(-1,1), pred.reshape(-1,1)))


In [ ]:
plt.figure(figsize=(10, 5))
dict_eth = {
    'Predictions': pred.reshape(1,-1)[0,20000:20050],
    'Close': re[0,20000:20050]
}
data_eth = pd.DataFrame(dict_eth)

plt.plot(data_eth[['Close', 'Predictions']])
plt.show()